# Compiling Fortran routine with a subroutine in a separate file using .pyf signature file to make it more pythonic

This code creates a python extension of a Fortran function which produces a fibonacci sequence and then adds one to all the entries. It is meant to test the use of subroutines which live in seprate files, as well as the use of signature files to make the input/output scheme more pythonic.

Lucas Myers

Written: July 16, 2020

### Create signature file

Note that `fibadd1.f` contains a subroutine `fibadd` which calls the original Fibonacci subroutine `fib` from the file `fib1.f`. First, we just need `fibadd1.f` to generate a `.pyf` signature file

In [1]:
import numpy.f2py

args = ['fibadd1.f', '-m', 'fibadd1', '-h', 'fibadd1.pyf']
failure = numpy.f2py.run_main(args)
print(failure)

Reading fortran codes...
	Reading file 'fibadd1.f' (format:fix,strict)
Post-processing...
	Block: fibadd1
			Block: fibadd
Post-processing (stage 2)...
Saving signatures to file ".\fibadd1.pyf"
None


### Print out contents of signature file

In [2]:
with open('fibadd1.pyf') as file:
    sig_file = file.read()
print(sig_file)

!    -*- f90 -*-
! Note: the context of this file is case sensitive.

python module fibadd1 ! in 
    interface  ! in :fibadd1
        subroutine fibadd(a,n) ! in :fibadd1:fibadd1.f
            real*8 dimension(n) :: a
            integer, optional,check(len(a)>=n),depend(a) :: n=len(a)
        end subroutine fibadd
    end interface 
end python module fibadd1

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/



### Modify signature file to make input/output scheme more pythonic and write to new file

In [3]:
modified_sig_file = \
"""!    -*- f90 -*-
python module fibadd2
    interface
        subroutine fibadd(a,n)
            real*8 dimension(n),depend(n),intent(out) :: a
            integer intent(in) :: n
        end subroutine fibadd
    end interface 
end python module fibadd2
"""

In [4]:
with open('fibadd2.pyf', 'w+') as file:
    file.write(modified_sig_file)

### Compile new, more pythonic Fortran code

To compile this, one reads in the source from the file containing the highest level subroutine `fibadd1.f` and then includes the lower level subroutine file `fib1.f` in the extra arguments. Additionally, to make the modifications dictated by the signature file, one must also include the name of the signature file in the extra arguments list. Also, make sure that the `modulename` keyword parameter in `numpy.f2py.compile()` matches the module name in the modified `.pyf` signature file.

In [5]:
with open('fibadd1.f') as file:
    source = file.read()
module = 'fibadd2'
args = ['fibadd2.pyf', 'fib1.f', '--compiler=mingw32']

failure = numpy.f2py.compile(source, extra_args=args, 
                             modulename=module, verbose=False)

print(failure)

0


### Test output

In [6]:
import fibadd2

a = fibadd2.fibadd(8)
print(a)

[ 1.  2.  2.  3.  4.  6.  9. 14.]
